In [9]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, log_loss, accuracy_score
from torch.utils.data import DataLoader, TensorDataset, random_split
import torchvision.datasets as datasets
import os
%matplotlib inline

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
import os
%matplotlib inline

In [10]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [ ]:
def load_data(Jd, l, num_conf, T, num_temps, batch_size, shuffle_opt, opt='train'):
    datasets = []
    spins_loaded = np.load(f'data_spins/{Jd}_{opt}/spins_{l}_{T[0]}_{T[-1]}.npz')
    answ_loaded = np.load(f'data_spins/{Jd}_{opt}/answ_{l}_{T[0]}_{T[-1]}.npz')
    for j in range(num_temps):
        
        #path = f'data_spins/{Jd}_{opt}/spins_{l}_{T[j]}.npy'
        #with open(path, 'rb') as f:
        #    x = np.load(f)   
        x = spins_loaded[f'T_{j}']
        tensor_x = torch.Tensor(x).unsqueeze(1)

        #path = f'data_spins/{Jd}_{opt}/answ_{l}_{T[j]}.npy'
        #with open(path, 'rb') as f:
        #    y = np.load(f)
        y = answ_loaded[f'T_{j}']
        tensor_y = torch.from_numpy(y).type(torch.float32)

        datasets.append(TensorDataset(tensor_x, tensor_y))


    dataset = torch.utils.data.ConcatDataset(datasets)

    return DataLoader(dataset, batch_size=batch_size, shuffle=shuffle_opt)

In [ ]:
class Net(nn.Module):
    def __init__(self, l):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 64, 2)
        self.pool = nn.MaxPool2d(2, 2)
        self.act_hid = nn.ReLU()
        self.fc1 = nn.Linear(64*int(l/2-1)*int(l/2-1), 64)
        self.fc2 = nn.Linear(64, 1)

    def forward(self, x):
        x = self.conv1(x)
        x = self.pool(x)
        x = self.act_hid(x)
        x = x.view(-1, 64*int(l/2-1)*int(l/2-1))
        x = self.fc1(x)
        x = self.act_hid(x)
        x = self.fc2(x)
        return x

In [ ]:
def train(l, train_dataloader, num_epoch, criterion, batch_size):
    model = Net(l)
    optimizer = torch.optim.Adam(model.parameters(), lr=1.0e-4)
    act = nn.Sigmoid()

    for epoch in range(num_epoch):  
        running_loss = 0.0
        accuracy = 0.0
        pbar = tqdm(enumerate(train_dataloader), total=len(train_dataloader))
        for i, data in pbar:
            inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.to(device)  
            
            model.to(device)
            
            optimizer.zero_grad()
            outputs = model(inputs)
            #outputs = act(outputs)

            outputs = outputs.squeeze(1) # к одной размерности с labels

            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            accuracy += (batch_size - sum(abs(labels - act(outputs)))).float().mean()

            pbar.set_description(
                    f"Loss: {running_loss/((i+1)*batch_size)} "
                    f"Accuracy: {accuracy * 100  / ((i+1)*batch_size)}"
            )

    print('Training completed')
    return model, optimizer

In [ ]:
def testing(model, test_dataloader, criterion, batch_size):
    outp = []
    errors = []
    accuracy = 0.0
    labels_all = []
    act = nn.Sigmoid()
    with torch.no_grad():
        for i, data in enumerate(test_dataloader):
            inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.to(device)
            model.to(device)
            outputs = model(inputs)
            #outputs = act(outputs)
            outputs = outputs.squeeze(1)
            outp.append(act(outputs).item())
            loss = criterion(outputs, labels)
            errors.append(loss.item())
            
            accuracy += sum(abs(labels - act(outputs))).float().mean()
            
            labels_all.append(labels.item())
    
    accuracy = (1 - accuracy / len(test_dataloader)) * 100
    mse = mean_squared_error(labels_all, outp, squared=False)
    logloss = log_loss(labels_all, outp)
    
    print("Accuracy = {}".format(accuracy))
    print("MSE = {}".format(mse))
    print("LogLoss = {}".format(logloss))
    return outp, errors, labels_all, accuracy, mse, logloss

In [11]:
roots = [2.2691853142129728, 2.104982167992544, 1.932307699120554, 1.749339162933206, 1.5536238493280832, 1.34187327905057, 1.109960313758399, 0.8541630993606272, 0.5762735442012712, 0.2885386111960936, 0.03198372863548067]
jds = [0.0, -0.1, -0.2, -0.3, -0.4, -0.5, -0.6, -0.7, -0.8, -0.9, -1.0]
get_crit_T = dict(zip(jds, roots))

In [ ]:
### training ###

def train_and_save(Jd, l, num_temps):
    num_conf_tr = 2048
    num_conf_ts = 512
    num_epoch = 1

    T_c = get_crit_T[Jd]
    T = np.round(np.linspace(T_c - 0.3, T_c + 0.3, num_temps), 4)
    #T = np.round(np.linspace(T_c - 10**-2.0, T_c + 10**-2.0, num_temps), 4)

    criterion = nn.BCEWithLogitsLoss()     

    train_dataloader = load_data(Jd, l, num_conf_tr, T, num_temps, batch_size=4, shuffle_opt=True, opt='train')
    print(f'Start training for L = {l}')
    model, optimizer = train(l, train_dataloader, num_epoch, criterion, batch_size=4)

    #PATH = f'models/{l}_{Jd}_{T[0]}_{T[-1]}_{num_temps}_{epoch}_epochs.pt'
    PATH = f'models/{l}_{Jd}_{T[0]}_{T[-1]}_{num_temps}_close_{epoch}_epochs.pt'
    torch.save(model.state_dict(), PATH)
    #PATH = f'models/{l}_{Jd}_{T[0]}_{T[-1]}_{num_temps}_{epoch}_epochs_opt.pt'
    PATH = f'models/{l}_{Jd}_{T[0]}_{T[-1]}_{num_temps}_close_{epoch}_epochs_opt.pt'
    torch.save(optimizer.state_dict(), PATH)

In [ ]:
L = [10]
Jd = 0.0
num_temps = 100
for l in L:
    train_and_save(Jd, l, num_temps)

In [ ]:
### testing ###

def get_errs_outs(Jd, l, num_temps):
    T_c = get_crit_T[Jd]
    #T = np.round(np.linspace(T_c - 0.3, T_c + 0.3, num_temps), 4)
    T = np.round(np.linspace(T_c - 10**-2.0, T_c + 10**-2.0, num_temps), 5)
    
    num_conf_tr = 2048
    num_conf_ts = 512

    criterion = nn.BCEWithLogitsLoss()     
    
    print(f'Start testing for L = {l}, Jd = {Jd}')
    model = Net(l)
    T_c_ = get_crit_T[0.0]
    T_ = np.round(np.linspace(T_c_ - 0.3, T_c_ + 0.3, num_temps), 4)
    #T_ = np.round(np.linspace(T_c_ - 10**-2.0, T_c_ + 10**-2.0, num_temps), 4)
    #PATH = f'models/{l}_0.0_{T_[0]}_{T_[-1]}_{num_temps}.pt'
    PATH = f'models/{l}_0.0_{T_[0]}_{T_[-1]}_{num_temps}_{num_epochs}_epochs.pt'

    model.load_state_dict(torch.load(PATH))
    model.eval()
    test_dataloader = load_data(Jd, l, num_conf_ts, T, num_temps, batch_size=1, shuffle_opt=False, opt='test')
    outp, errors, labels, accuracy, mse, logloss = testing(model, test_dataloader, criterion, batch_size=1)
    return errors, outp, labels, accuracy, mse, logloss

In [ ]:
metrics = {}

num_epochs = 5
L = [60, 80]
for l in L:
    metrics[l] = []
#Jds = [0.0, -0.1, -0.2, -0.3, -0.4, -0.5, -0.6, -0.7, -0.8, -0.9]
Jds = [-0.5]
num_temps = 100
for Jd in Jds: 
    for l in L:
        errs_outs = get_errs_outs(Jd, l, num_temps)
        metrics[l].append([Jd, errs_outs[3], errs_outs[4], errs_outs[5]])
        #np.save(f'data_errors/{Jd}_{l}_{num_temps}_{num_epochs}_epochs.npy', errs_outs[0])
        np.save(f'data_errors/{Jd}_{l}_{num_temps}_{num_epochs}_epochs_close.npy', errs_outs[0])
        #np.save(f'data_errors/{Jd}_{l}_{num_temps}.npy', errs_outs[0])
        #np.save(f'data_outputs/{Jd}_{l}_{num_temps}_{num_epochs}_epochs.npy', errs_outs[1])
        np.save(f'data_outputs/{Jd}_{l}_{num_temps}_{num_epochs}_epochs_close.npy', errs_outs[1])
        #np.save(f'data_outputs/{Jd}_{l}_{num_temps}.npy', errs_outs[1])

In [3]:
l = 100
Jd = 1.0
num_temps = 100
roots = [0.3018336135076354, 0.7883671029813581, 0.9723445185469877, 1.2390777517571931, 1.6410179299284857, 1.9728374883141215, 2.2691853142129728, 2.8007227202811613]
ms = [0.0004, 0.0625, 0.125, 0.25, 0.5, 0.75, 1.0, 1.5]
get_crit_T = dict(zip(ms, roots))
T_c = get_crit_T[Jd]
T = np.round(np.linspace(T_c - 0.3, T_c + 0.3, num_temps), 4)
#T = np.round(np.linspace(T_c - 10**-2.0, T_c + 10**-2.0, num_temps), 5)
#T = np.round(np.linspace(0.03, 3.5, num_temps), 4)

if Jd == 1.0:
    opt = 'train'
    T_miss = []
    for j in range(num_temps):
            path = f'data_spins/{Jd}_{opt}/spins_{l}_{T[j]}.npy'
            if not os.path.isfile(path):
                T_miss.append(T[j])
    print(T_miss, len(T_miss))
    path = f'data_spins/{Jd}_{opt}/spins_{l}_{T[0]}_{T[-1]}.npz'
    if not os.path.isfile(path):
        print('\nNo NPZ')
    else:
        print('\nNPZ is available')

opt = 'test'
T_miss = []
for j in range(num_temps):
        path = f'data_spins/{Jd}_{opt}/spins_{l}_{T[j]}.npy'
        if not os.path.isfile(path):
            T_miss.append(T[j])
print(T_miss, len(T_miss))

path = f'data_spins/{Jd}_{opt}/spins_{l}_{T[0]}_{T[-1]}.npz'
if not os.path.isfile(path):
    print('\nNo NPZ')
else:
    print('\nNPZ is available')

[] 0

No NPZ
[1.9692, 1.9752, 1.9813, 1.9874, 1.9934, 1.9995, 2.0055, 2.0116, 2.0177, 2.0237, 2.0298, 2.0359, 2.0419, 2.048, 2.054, 2.0601, 2.0662, 2.0722, 2.0783, 2.0843, 2.0904, 2.0965, 2.1025, 2.1086, 2.1146, 2.1207, 2.1268, 2.1328, 2.1389, 2.1449, 2.151, 2.1571, 2.1631, 2.1692, 2.1752, 2.1813, 2.1874, 2.1934, 2.1995, 2.2055, 2.2116, 2.2177, 2.2237, 2.2298, 2.2359, 2.2419, 2.248, 2.254, 2.2601, 2.2662, 2.2722, 2.2783, 2.2843, 2.2904, 2.2965, 2.3025, 2.3086, 2.3146, 2.3207, 2.3268, 2.3328, 2.3389, 2.3449, 2.351, 2.3571, 2.3631, 2.3692, 2.3752, 2.3813, 2.3874, 2.3934, 2.3995, 2.4055, 2.4116, 2.4177, 2.4237, 2.4298, 2.4359, 2.4419, 2.448, 2.454, 2.4601, 2.4662, 2.4722, 2.4783, 2.4843, 2.4904, 2.4965, 2.5025, 2.5086, 2.5146, 2.5207, 2.5268, 2.5328, 2.5389, 2.5449, 2.551, 2.5571, 2.5631, 2.5692] 100

NPZ is available


In [ ]:
from torchsummary import summary

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # PyTorch v0.4.0
l = 20
model = Net(l).to(device)

summary(model, (1, l, l))

In [ ]:
spins_loaded = np.load(f'data_spins/{Jd}_{opt}/spins_{l}_{T[0]}_{T[-1]}.npz')
#answ_loaded = np.load(f'data_spins/{Jd}_{opt}/answ_{l}_{T[0]}_{T[-1]}.npz')
answ_loaded = np.load(f'data_spins/{Jd}_{opt}/answ_{l}_{T[0]}_{T[-1]}_modified.npz')
for j in range(num_temps):
    
    #path = f'data_spins/{Jd}_{opt}/spins_{l}_{T[j]}.npy'
    #with open(path, 'rb') as f:
    #    x = np.load(f)   
    x = spins_loaded[f'T_{j}']
    
    #path = f'data_spins/{Jd}_{opt}/answ_{l}_{T[j]}.npy'
    #with open(path, 'rb') as f:
    #    y = np.load(f)
    y = answ_loaded[f'T_{j}']

In [13]:
%%time

#### !!!! только для -0.9 !!!! 

L = [20,30,60,80,120]
#Jds = [0.0, -0.1, -0.2, -0.3, -0.4, -0.5, -0.6, -0.7, -0.8]
#Jds = [-0.1, -0.2, -0.3, -0.4, -0.5]
Jds = [-0.9]
opt = 'test'
num_temps = 100

for l in L:
    for Jd in Jds:
        print(f'Start compressing for L = {l}, Jd = {Jd}')

        T_c = get_crit_T[Jd]
        T = np.round(np.linspace(T_c - 0.3, T_c + 0.3, num_temps), 4)
        #T = np.round(np.linspace(0.03, 3.5, num_temps), 4)
        #T = np.round(np.linspace(T_c - 10**-2.0, T_c + 10**-2.0, num_temps), 5)

        xs = []
        spins_loaded = np.load(f'data_spins/{Jd}_{opt}/spins_{l}_{T[0]}_{T[-1]}.npz')
        for j in range(2, num_temps): #### !!!! убираем T<0
            x_j = spins_loaded[f'T_{j}']
            xs.append(x_j)
        
        print(len(xs))
            
        savez_dict = dict()
        for j, x_j in enumerate(xs):
            savez_dict[f'T_{j}'] = x_j

        np.savez_compressed(f'data_spins/{Jd}_{opt}/spins_{l}_{T[0]}_{T[-1]}_upd', **savez_dict)

        ys = []
        answ_loaded = np.load(f'data_spins/{Jd}_{opt}/answ_{l}_{T[0]}_{T[-1]}_modified.npz')
        for j in range(2, num_temps):
            y_j = answ_loaded[f'T_{j}']
            ys.append(y_j)
            
        savez_dict = dict()
        for j, y_j in enumerate(ys):
            savez_dict[f'T_{j}'] = y_j

        np.savez_compressed(f'data_spins/{Jd}_{opt}/answ_{l}_{T[0]}_{T[-1]}_modified_upd', **savez_dict)

Start compressing for L = 20, Jd = -0.9
98
Start compressing for L = 30, Jd = -0.9
98
Start compressing for L = 60, Jd = -0.9
98
Start compressing for L = 80, Jd = -0.9
98
Start compressing for L = 120, Jd = -0.9
98
CPU times: user 2min 18s, sys: 9.17 s, total: 2min 27s
Wall time: 2min 28s


In [3]:
%%time

#L = [20, 30, 60, 80]
L = [120]
#Jds = [0.0, -0.1, -0.2, -0.3, -0.4, -0.5, -0.6, -0.7, -0.8]
#Jds = [-0.1, -0.2, -0.3, -0.4, -0.5]
#Jds = [1.0]
Jds = [0.0625, 0.125]#, 0.25, 0.5, 0.75, 1.0, 1.5]
opt = 'test'
num_temps = 100

for l in L:
    for Jd in Jds:

        #Jd = np.round(Jd, 2)
        print(f'Start compressing for L = {l}, Jd = {Jd}')

        roots = [0.3018336135076354, 0.7883671029813581, 0.9723445185469877, 1.2390777517571931, 1.6410179299284857, 1.9728374883141215, 2.2691853142129728, 2.8007227202811613]
        ms = [0.0004, 0.0625, 0.125, 0.25, 0.5, 0.75, 1.0, 1.5]
        get_crit_T = dict(zip(ms, roots))
        T_c = get_crit_T[Jd]
        T = np.round(np.linspace(T_c - 0.3, T_c + 0.3, num_temps), 4)
        #T = np.round(np.linspace(0.03, 3.5, num_temps), 4)
        #T = np.round(np.linspace(T_c - 10**-2.0, T_c + 10**-2.0, num_temps), 5)

        xs = []
        for j in range(num_temps):
            path = f'data_spins/{Jd}_{opt}/spins_{l}_{T[j]}.npy'
            with open(path, 'rb') as f:
                x_j = np.load(f)
                xs.append(x_j)
            os.remove(path)

        savez_dict = dict()
        for j, x_j in enumerate(xs):
            savez_dict[f'T_{j}'] = x_j

        np.savez_compressed(f'data_spins/{Jd}_{opt}/spins_{l}_{T[0]}_{T[-1]}', **savez_dict)

        ys = []
        for j in range(num_temps):
            path = f'data_spins/{Jd}_{opt}/answ_{l}_{T[j]}.npy'
            with open(path, 'rb') as f:
                y_j = np.load(f)
                ys.append(y_j)
            os.remove(path)

        savez_dict = dict()
        for j, y_j in enumerate(ys):
            savez_dict[f'T_{j}'] = y_j

        np.savez_compressed(f'data_spins/{Jd}_{opt}/answ_{l}_{T[0]}_{T[-1]}', **savez_dict)

Start compressing for L = 120, Jd = 0.0625
Start compressing for L = 120, Jd = 0.125
CPU times: user 2min 14s, sys: 14 s, total: 2min 28s
Wall time: 2min 51s


In [ ]:
l = 80
Jd = 0.0
T_c = get_crit_T[Jd]
T = np.round(np.linspace(T_c - 0.3, T_c + 0.3, num_temps), 4)

loaded = np.load(f'data_spins/{Jd}_{opt}/spins_{l}_{T[0]}_{T[-1]}.npz')
loaded['T_0'] == loaded['T_69']

In [ ]:
total_size = 0.0

#L = [10, 20, 30, 60, 80]
L = [80]
Jd = 0.0
opt = 'test'
num_temps = 100

T_c = get_crit_T[Jd]
T = np.round(np.linspace(T_c - 0.3, T_c + 0.3, num_temps), 4)
#spins_loaded = np.load(f'data_spins/{Jd}_{opt}/spins_{l}_{T[0]}_{T[-1]}.npz')
#answ_loaded = np.load(f'data_spins/{Jd}_{opt}/answ_{l}_{T[0]}_{T[-1]}.npz')
for j in range(num_temps):
    for l in L:
        path = f'data_spins/{Jd}_{opt}/spins_{l}_{T[j]}.npy'
        with open(path, 'rb') as f:
            x = np.load(f)   
        total_size += x.nbytes
        #x = spins_loaded[f'T_{j}']
        #tensor_x = torch.Tensor(x).unsqueeze(1)
print(total_size / (1024 * 1024 * 1024))
#round(getsizeof(a) / 1024 / 1024,2)

In [ ]:
l = 80
Jd = 0.0
T_c = get_crit_T[Jd]
T = np.round(np.linspace(T_c - 0.3, T_c + 0.3, num_temps), 4)

loaded = np.load(f'data_spins/{Jd}_{opt}/spins_{l}_{T[0]}_{T[-1]}.npz')
loaded['T_0'].nbytes / (1024 * 1024)

In [ ]:
%%time

from zipfile import ZipFile, ZIP_LZMA, ZIP_BZIP2

L = [80]
Jds = [0.0]
opt = 'train'
num_temps = 100

for l in L:
    for Jd in Jds:
        
        print(f'Start compressing for L = {l}, Jd = {Jd}')
        
        T_c = get_crit_T[Jd]
        T = np.round(np.linspace(T_c - 0.3, T_c + 0.3, num_temps), 4)
        #T = np.round(np.linspace(0.03, 3.5, num_temps), 4)

        xs = []
        for j in range(num_temps):
            path = f'data_spins/{Jd}_{opt}/spins_{l}_{T[j]}.npy'
            with open(path, 'rb') as f:
                x_j = np.load(f)
                xs.append(x_j)

        #savez_dict = dict()
        #for j, x_j in enumerate(xs):
        #    savez_dict[f'T_{j}'] = x_j

        #np.savez_compressed(f'data_spins/{Jd}_{opt}/spins_{l}_{T[0]}_{T[-1]}', **savez_dict)
        
        np.save(f'data_spins/{Jd}_{opt}/spins_{l}_{T[0]}_{T[-1]}.npy', xs)

        with ZipFile(f'data_spins/{Jd}_{opt}/spins_{l}_{T[0]}_{T[-1]}_bzip.zip', 'w', compression=ZIP_BZIP2) as myzip:
            myzip.write(f'data_spins/{Jd}_{opt}/spins_{l}_{T[0]}_{T[-1]}.npy')
        
        myzip.close()
        
        ys = []
        for j in range(num_temps):
            path = f'data_spins/{Jd}_{opt}/answ_{l}_{T[j]}.npy'
            with open(path, 'rb') as f:
                y_j = np.load(f)
                ys.append(y_j)

        #savez_dict = dict()
        #for j, y_j in enumerate(ys):
        #    savez_dict[f'T_{j}'] = y_j

        #np.savez_compressed(f'data_spins/{Jd}_{opt}/answ_{l}_{T[0]}_{T[-1]}', **savez_dict)
        
        np.save(f'data_spins/{Jd}_{opt}/answ_{l}_{T[0]}_{T[-1]}.npy', ys)
        
        with ZipFile(f'data_spins/{Jd}_{opt}/answ_{l}_{T[0]}_{T[-1]}_bzip.zip', 'w', compression=ZIP_BZIP2) as myzip:
            myzip.write(f'data_spins/{Jd}_{opt}/answ_{l}_{T[0]}_{T[-1]}.npy')
        myzip.close()

In [ ]:
%%time

from zipfile import ZipFile, ZIP_LZMA, ZIP_BZIP2

L = [80]
Jds = [0.0]
opt = 'train'
num_temps = 100

for l in L:
    for Jd in Jds:
        
        print(f'Start compressing for L = {l}, Jd = {Jd}')
        
        T_c = get_crit_T[Jd]
        T = np.round(np.linspace(T_c - 0.3, T_c + 0.3, num_temps), 4)
        #T = np.round(np.linspace(0.03, 3.5, num_temps), 4)

        xs = []
        for j in range(num_temps):
            path = f'data_spins/{Jd}_{opt}/spins_{l}_{T[j]}.npy'
            with open(path, 'rb') as f:
                x_j = np.load(f)
                xs.append(x_j)

        #savez_dict = dict()
        #for j, x_j in enumerate(xs):
        #    savez_dict[f'T_{j}'] = x_j

        #np.savez_compressed(f'data_spins/{Jd}_{opt}/spins_{l}_{T[0]}_{T[-1]}', **savez_dict)
        
        np.save(f'data_spins/{Jd}_{opt}/spins_{l}_{T[0]}_{T[-1]}.npy', xs)

        with ZipFile(f'data_spins/{Jd}_{opt}/spins_{l}_{T[0]}_{T[-1]}_lzma.zip', 'w', compression=ZIP_LZMA) as myzip:
            myzip.write(f'data_spins/{Jd}_{opt}/spins_{l}_{T[0]}_{T[-1]}.npy')
        
        myzip.close()
        
        ys = []
        for j in range(num_temps):
            path = f'data_spins/{Jd}_{opt}/answ_{l}_{T[j]}.npy'
            with open(path, 'rb') as f:
                y_j = np.load(f)
                ys.append(y_j)

        #savez_dict = dict()
        #for j, y_j in enumerate(ys):
        #    savez_dict[f'T_{j}'] = y_j

        #np.savez_compressed(f'data_spins/{Jd}_{opt}/answ_{l}_{T[0]}_{T[-1]}', **savez_dict)
        
        np.save(f'data_spins/{Jd}_{opt}/answ_{l}_{T[0]}_{T[-1]}.npy', ys)
        
        with ZipFile(f'data_spins/{Jd}_{opt}/answ_{l}_{T[0]}_{T[-1]}_lzma.zip', 'w', compression=ZIP_LZMA) as myzip:
            myzip.write(f'data_spins/{Jd}_{opt}/answ_{l}_{T[0]}_{T[-1]}.npy')
        myzip.close()

In [15]:
spins_loaded = np.load(f'data_spins/{Jd}_{opt}/spins_{l}_{T[0]}_{T[-1]}_upd.npz')
for j in range(num_temps-2): #### !!!! убираем T<0
    if spins_loaded[f'T_{j}'] is not None:
       print(f'T_{j} is OK')

T_0 is OK
T_1 is OK
T_2 is OK
T_3 is OK
T_4 is OK
T_5 is OK
T_6 is OK
T_7 is OK
T_8 is OK
T_9 is OK
T_10 is OK
T_11 is OK
T_12 is OK
T_13 is OK
T_14 is OK
T_15 is OK
T_16 is OK
T_17 is OK
T_18 is OK
T_19 is OK
T_20 is OK
T_21 is OK
T_22 is OK
T_23 is OK
T_24 is OK
T_25 is OK
T_26 is OK
T_27 is OK
T_28 is OK
T_29 is OK
T_30 is OK
T_31 is OK
T_32 is OK
T_33 is OK
T_34 is OK
T_35 is OK
T_36 is OK
T_37 is OK
T_38 is OK
T_39 is OK
T_40 is OK
T_41 is OK
T_42 is OK
T_43 is OK
T_44 is OK
T_45 is OK
T_46 is OK
T_47 is OK
T_48 is OK
T_49 is OK
T_50 is OK
T_51 is OK
T_52 is OK
T_53 is OK
T_54 is OK
T_55 is OK
T_56 is OK
T_57 is OK
T_58 is OK
T_59 is OK
T_60 is OK
T_61 is OK
T_62 is OK
T_63 is OK
T_64 is OK
T_65 is OK
T_66 is OK
T_67 is OK
T_68 is OK
T_69 is OK
T_70 is OK
T_71 is OK
T_72 is OK
T_73 is OK
T_74 is OK
T_75 is OK
T_76 is OK
T_77 is OK
T_78 is OK
T_79 is OK
T_80 is OK
T_81 is OK
T_82 is OK
T_83 is OK
T_84 is OK
T_85 is OK
T_86 is OK
T_87 is OK
T_88 is OK
T_89 is OK
T_90 is OK
T_91 is O